In [1]:
"""
Game Semantics Simulator (Executable Python Module)

This module implements a simplified, yet extensible, simulation framework for
Game Semantics where logical proofs are modeled as interactive games between
Prover (Player/P) and Refuter (Opponent/O). It includes two example logical
frameworks (Linear Logic and Classical Logic) as game rulesets, definitions of
arenas, moves, positions, strategies, and evaluation tools to test legality,
determinacy, and (bounded) completeness of strategies.

Design goals:
- Educational and executable: runnable from the command line to try example
  games and strategies.
- Extensible: add new logic frameworks by subclassing LogicFramework and
  defining legality/resource rules.
- Rigorous style: docstrings, type hints, and clear separation of concerns.

Limitations:
- This is a pedagogical simulator, not a full formal semantics engine. The
  semantics are simplified to keep the implementation compact while preserving
  key ideas: moves, alternation, resources (linear), and duplication (classical).

Structure (hierarchical exposition embedded in docstrings):
1) Foundations: Players, Moves, Arenas, and Histories (positions).
2) Logical Frameworks: abstract LogicFramework + LinearLogicFramework and
   ClassicalLogicFramework instantiations (legality rules and resource model).
3) Strategies: deterministic and nondeterministic strategies modeled as
   functions from history -> set of legal moves (or single move for deterministic).
4) Analysis: bounded completeness checks (strategy wins against all opponent
   continuations up to depth), determinacy testing (search for winning strategy),
   and legality validators.
5) Examples: sample arenas for implication, conjunction, and a small classical
   tautology. Example strategies for Prover and Refuter.

Run:
    python game_semantics.py

"""
from __future__ import annotations
from typing import List, Tuple, Callable, Dict, Optional, Set, Iterable
from enum import Enum, auto
import random
import itertools
import sys
import copy

# ----------------------
# Foundational definitions
# ----------------------

class Player(Enum):
    PROVER = "P"  # Player / Prover
    REFUTER = "O"  # Opponent / Refuter

    def other(self) -> "Player":
        return Player.PROVER if self is Player.REFUTER else Player.REFUTER


class Move:
    """A single move in the arena.

    Attributes:
        label: symbolic label of the move (e.g., 'q', 'a1', 'attack').
        player: which player is allowed to make this move (P or O). Note that
                legality will further constrain moves based on history and
                the logic framework.
        resource_id: optional token name used by linear logic to track resource
                     consumption. If None, move is not resource-sensitive.
    """

    def __init__(self, label: str, player: Player, resource_id: Optional[str] = None):
        self.label = label
        self.player = player
        self.resource_id = resource_id

    def __repr__(self) -> str:
        return f"Move({self.label!r},{self.player.value}{','+self.resource_id if self.resource_id else ''})"

    def __eq__(self, other) -> bool:
        return isinstance(other, Move) and (self.label, self.player, self.resource_id) == (other.label, other.player, other.resource_id)

    def __hash__(self):
        return hash((self.label, self.player, self.resource_id))


Position = Tuple[Move, ...]  # history / sequence of moves


class Arena:
    """An arena defines possible initial moves and how moves enable others.

    For simplicity the arena is represented by:
      - initial_moves: set of moves that may start the play
      - enable_map: mapping from a move label to set of moves that it enables

    This is a simplification of full arena graphs used in formal game semantics.
    """

    def __init__(self, initial_moves: Iterable[Move], enable_map: Optional[Dict[str, Set[Move]]] = None):
        self.initial_moves: Set[Move] = set(initial_moves)
        self.enable_map: Dict[str, Set[Move]] = enable_map if enable_map is not None else {}

    def enabled_after(self, history: Position) -> Set[Move]:
        """Return the set of moves enabled given the history.

        Rules:
          - If history is empty: initial moves are enabled.
          - Otherwise, enabled moves are union of enable_map for last move label.
        """
        if len(history) == 0:
            return set(self.initial_moves)
        last_label = history[-1].label
        return set(self.enable_map.get(last_label, set()))


# ----------------------
# Logic frameworks
# ----------------------

class LogicFramework:
    """Abstract base class defining legality/resource rules for a logic.

    Implementations must define `is_legal_move` and may override
    `initial_state_resources` and `update_resources_after_move`.
    """

    def __init__(self, arena: Arena):
        self.arena = arena

    def initial_state_resources(self) -> Dict[str, int]:
        """Return initial resource multiset. Default: empty (no resources).

        For linear logic, moves with resource_id create tokens in arena design.
        """
        return {}

    def is_legal_move(self, history: Position, move: Move, resources: Dict[str, int]) -> bool:
        """Check if move is legal under current history and resource multiset.

        Default: a move is legal iff it is enabled by the arena and the move's
        declared player matches the expected player (alternating play).
        """
        # enabled by arena
        enabled = move in self.arena.enabled_after(history)
        if not enabled:
            return False
        # alternation: players alternate moves
        if len(history) == 0:
            expected = Player.PROVER  # convention: Prover starts
        else:
            expected = history[-1].player.other()
        if move.player != expected:
            return False
        # resources: subclass may inspect `resources`
        return True

    def update_resources_after_move(self, resources: Dict[str, int], move: Move) -> Dict[str, int]:
        """Return new resource multiset after applying move. Default: no change."""
        return dict(resources)


class LinearLogicFramework(LogicFramework):
    """Linear logic framework: resources are consumed when a move uses a resource_id.

    Contraction and weakening are forbidden: resource tokens are single-use.
    """

    def initial_state_resources(self) -> Dict[str, int]:
        # collect resource ids from arena moves and give one token each
        res = {}
        all_moves = set(self.arena.initial_moves) | set(itertools.chain.from_iterable(self.arena.enable_map.values()))
        for m in all_moves:
            if m.resource_id:
                res.setdefault(m.resource_id, 0)
                res[m.resource_id] += 1
        return res

    def is_legal_move(self, history: Position, move: Move, resources: Dict[str, int]) -> bool:
        if not super().is_legal_move(history, move, resources):
            return False
        if move.resource_id:
            return resources.get(move.resource_id, 0) > 0
        return True

    def update_resources_after_move(self, resources: Dict[str, int], move: Move) -> Dict[str, int]:
        res = dict(resources)
        if move.resource_id:
            res[move.resource_id] = res.get(move.resource_id, 0) - 1
            if res[move.resource_id] <= 0:
                del res[move.resource_id]
        return res


class ClassicalLogicFramework(LogicFramework):
    """Classical logic framework: allows duplication/weakening.

    For our simplified model, moves with resource_id do not consume tokens, and
    any move may be repeated (no resource restriction).
    """

    def initial_state_resources(self) -> Dict[str, int]:
        return {}

    # is_legal_move and update_resources_after_move inherit the defaults


# ----------------------
# Strategies
# ----------------------

Strategy = Callable[[Position, LogicFramework, Dict[str, int]], Optional[Move]]
# Deterministic mapping from history and current resource state to a move (or None)


def random_strategy_factory(seed: Optional[int] = None) -> Strategy:
    rng = random.Random(seed)

    def strat(history: Position, logic: LogicFramework, resources: Dict[str, int]) -> Optional[Move]:
        enabled = [m for m in logic.arena.enabled_after(history) if logic.is_legal_move(history, m, resources)]
        if not enabled:
            return None
        return rng.choice(enabled)

    return strat


class DeterministicStrategy:
    """A simple deterministic strategy built from a decision table.

    decision_table: mapping from (history labels tuple) -> move label to choose.
    If a mapping is absent, you may return None or select a legal default.
    """

    def __init__(self, decision_table: Dict[Tuple[str, ...], str]):
        self.table = decision_table

    def __call__(self, history: Position, logic: LogicFramework, resources: Dict[str, int]) -> Optional[Move]:
        key = tuple(m.label for m in history)
        desired = self.table.get(key)
        enabled = logic.arena.enabled_after(history)
        legal = [m for m in enabled if logic.is_legal_move(history, m, resources)]
        if desired:
            for m in legal:
                if m.label == desired:
                    return m
        # fallback: first legal move
        return legal[0] if legal else None


# ----------------------
# Game execution and analysis
# ----------------------

class GameResult(Enum):
    PROVER_WIN = auto()
    REFUTER_WIN = auto()
    DRAW = auto()


def play_game(prover: Strategy, refuter: Strategy, logic: LogicFramework, max_depth: int = 20, verbose: bool = False) -> Tuple[GameResult, Position]:
    """Play a single game deterministically using the given strategies.

    Players alternate starting with Prover by convention. The game ends when a
    player has no legal moves (they lose), or when max_depth reached (draw).
    """
    history: List[Move] = []
    resources = logic.initial_state_resources()
    for step in range(max_depth):
        current = Player.PROVER if step % 2 == 0 else Player.REFUTER
        strat = prover if current is Player.PROVER else refuter
        move = strat(tuple(history), logic, dict(resources))
        if move is None or not logic.is_legal_move(tuple(history), move, resources):
            # current player cannot move -> they lose
            winner = GameResult.REFUTER_WIN if current is Player.PROVER else GameResult.PROVER_WIN
            if verbose:
                print(f"Step {step}: {current.value} cannot move. Winner: {winner}")
            return winner, tuple(history)
        history.append(move)
        resources = logic.update_resources_after_move(resources, move)
        if verbose:
            print(f"Step {step}: {current.value} plays {move}. Resources now {resources}")
    # reached max depth
    return GameResult.DRAW, tuple(history)


def all_possible_moves_for_player(history: Position, logic: LogicFramework) -> List[Move]:
    resources = logic.initial_state_resources()
    # Simulate resource consumption along history to get current resources
    for mv in history:
        resources = logic.update_resources_after_move(resources, mv)
    return [m for m in logic.arena.enabled_after(history) if logic.is_legal_move(history, m, resources)]


def exhaustive_win_check(player: Player, logic: LogicFramework, max_depth: int = 6) -> Tuple[bool, Optional[Strategy]]:
    """Brute-force check whether `player` has a winning strategy in the arena up to max_depth.

    Returns (has_winning_strategy, strategy_as_decision_table_if_found).
    This constructs a full decision table for the player mapping from history->winning move if exists.

    Note: exponential and only for small arenas / depths.
    """
    # We'll attempt to construct a strategy by backward induction over finite plays.

    initial_resources = logic.initial_state_resources()

    # Build game tree nodes: (history, resources)
    # For small depth, recursively search for a winning move for `player`.

    from functools import lru_cache

    @lru_cache(maxsize=None)
    def node_winner(history_tuple: Tuple[Tuple[str, str], ...], resources_tuple: Tuple[Tuple[str, int], ...], turn: Player, depth: int) -> bool:
        # returns True iff `player` can force a win from this node
        if depth == 0:
            return False
        # reconstruct history and resources
        history = tuple(Move(label, Player(p)) for (label, p) in history_tuple)
        resources = dict(resources_tuple)
        enabled = [m for m in logic.arena.enabled_after(history) if logic.is_legal_move(history, m, resources)]
        if not enabled:
            # current player cannot move -> they lose
            return turn is not player
        if turn is player:
            # there exists a move leading to states where player forces win
            for m in enabled:
                new_resources = logic.update_resources_after_move(resources, m)
                new_hist = history + (m,)
                rt = node_winner(tuple((mv.label, mv.player.value) for mv in new_hist), tuple(sorted(new_resources.items())), turn.other(), depth - 1)
                if rt:
                    return True
            return False
        else:
            # opponent moves: player must win for all opponent choices
            for m in enabled:
                new_resources = logic.update_resources_after_move(resources, m)
                new_hist = history + (m,)
                rt = node_winner(tuple((mv.label, mv.player.value) for mv in new_hist), tuple(sorted(new_resources.items())), turn.other(), depth - 1)
                if not rt:
                    return False
            return True

    # initial node
    history0: Tuple[Tuple[str, str], ...] = tuple()
    resources0 = tuple(sorted(initial_resources.items()))
    starting_player = Player.PROVER  # by convention
    result = node_winner(history0, resources0, starting_player, max_depth)
    return result, None


def is_strategy_complete(strategy: Strategy, player: Player, logic: LogicFramework, max_depth: int = 6) -> bool:
    """Check (bounded) completeness: strategy wins against all opponent strategies up to depth.

    For every possible opponent play, if following the player's strategy leads to a win, it's complete.
    """
    initial_resources = logic.initial_state_resources()

    # We'll enumerate opponent choices nondeterministically (exhaustively) while fixing player's moves to strategy
    from functools import lru_cache

    @lru_cache(maxsize=None)
    def node_is_win(history_tuple: Tuple[Tuple[str, str], ...], resources_tuple: Tuple[Tuple[str, int], ...], depth: int) -> bool:
        if depth == 0:
            return False
        history = tuple(Move(label, Player(p)) for (label, p) in history_tuple)
        resources = dict(resources_tuple)
        turn = Player.PROVER if len(history) % 2 == 0 else Player.REFUTER
        if turn is player:
            # follow strategy
            mv = strategy(history, logic, dict(resources))
            if mv is None or not logic.is_legal_move(history, mv, resources):
                return False
            new_resources = logic.update_resources_after_move(resources, mv)
            new_hist = history + (mv,)
            return node_is_win(tuple((m.label, m.player.value) for m in new_hist), tuple(sorted(new_resources.items())), depth - 1)
        else:
            # opponent may choose any legal move; player's strategy must survive all
            enabled = [m for m in logic.arena.enabled_after(history) if logic.is_legal_move(history, m, resources)]
            if not enabled:
                # opponent cannot move -> player wins
                return True
            for m in enabled:
                new_resources = logic.update_resources_after_move(resources, m)
                new_hist = history + (m,)
                if not node_is_win(tuple((mv.label, mv.player.value) for mv in new_hist), tuple(sorted(new_resources.items())), depth - 1):
                    return False
            return True

    return node_is_win(tuple(), tuple(sorted(initial_resources.items())), max_depth)


# ----------------------
# Small example arenas and strategies
# ----------------------

def make_implication_arena() -> Arena:
    """Arena representing a simple implication game for "A -> B" where

    - Prover starts by asserting the implication (P: 'imp')
    - Refuter may challenge by asking for A (O: 'askA')
    - Prover must provide proof of A (P: 'giveA') which then allows Prover to produce B (P: 'giveB')

    Moves are decorated with resource ids to illustrate linear/resource use.
    """
    imp = Move('imp', Player.PROVER)
    askA = Move('askA', Player.REFUTER, resource_id='A')
    giveA = Move('giveA', Player.PROVER, resource_id='A')
    giveB = Move('giveB', Player.PROVER)
    enable_map = {
        'imp': {askA},
        'askA': {giveA},
        'giveA': {giveB},
    }
    return Arena([imp], enable_map)


def make_conjunction_arena() -> Arena:
    """Arena representing conjunction A & B proof game.

    Prover must present both components A and B when challenged.
    """
    start = Move('start', Player.PROVER)
    challengeA = Move('challengeA', Player.REFUTER, resource_id='A')
    challengeB = Move('challengeB', Player.REFUTER, resource_id='B')
    provideA = Move('provideA', Player.PROVER, resource_id='A')
    provideB = Move('provideB', Player.PROVER, resource_id='B')
    enable_map = {
        'start': {challengeA, challengeB},
        'challengeA': {provideA},
        'challengeB': {provideB},
        'provideA': set(),
        'provideB': set(),
    }
    return Arena([start], enable_map)


def simple_prover_strategy(history: Position, logic: LogicFramework, resources: Dict[str, int]) -> Optional[Move]:
    # Always play the first legal move available
    enabled = [m for m in logic.arena.enabled_after(history) if logic.is_legal_move(history, m, resources)]
    return enabled[0] if enabled else None


def simple_refuter_strategy(history: Position, logic: LogicFramework, resources: Dict[str, int]) -> Optional[Move]:
    # Prefer challenging moves if possible
    enabled = [m for m in logic.arena.enabled_after(history) if logic.is_legal_move(history, m, resources)]
    if not enabled:
        return None
    for m in enabled:
        if m.player is Player.REFUTER:
            return m
    return enabled[0]


# ----------------------
# Command-line demo
# ----------------------

def demo_implication(logic_cls=LinearLogicFramework):
    arena = make_implication_arena()
    logic = logic_cls(arena)
    prover = simple_prover_strategy
    refuter = simple_refuter_strategy
    print(f"Demo implication arena under {logic_cls.__name__}")
    res, history = play_game(prover, refuter, logic, max_depth=10, verbose=True)
    print("Result:", res, "History:", history)


def demo_conjunction(logic_cls=LinearLogicFramework):
    arena = make_conjunction_arena()
    logic = logic_cls(arena)
    prover = simple_prover_strategy
    refuter = simple_refuter_strategy
    print(f"Demo conjunction arena under {logic_cls.__name__}")
    res, history = play_game(prover, refuter, logic, max_depth=10, verbose=True)
    print("Result:", res, "History:", history)


def run_small_experiments():
    print("=== Linear Logic implication demo ===")
    demo_implication(LinearLogicFramework)
    print("\n=== Classical Logic implication demo ===")
    demo_implication(ClassicalLogicFramework)
    print("\n=== Linear Logic conjunction demo ===")
    demo_conjunction(LinearLogicFramework)
    print("\n=== Exhaustive determinacy check (bounded) on conjunction arena ===")
    arena = make_conjunction_arena()
    logic = LinearLogicFramework(arena)
    has_win, _ = exhaustive_win_check(Player.PROVER, logic, max_depth=6)
    print("Prover has winning strategy (bounded):", has_win)


if __name__ == '__main__':
    # Basic CLI to run demos
    if len(sys.argv) > 1 and sys.argv[1] == 'demo':
        run_small_experiments()
    else:
        print("Game Semantics simulator module. Run with `python game_semantics.py demo` to execute examples.")

# End of module


Game Semantics simulator module. Run with `python game_semantics.py demo` to execute examples.
